# импорты

In [218]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter

my_stopwords = [] #заглушка если придумаем стопслова
nltk.download('stopwords')
nltk.download('wordnet')
russian_stopwords_new = stopwords.words("russian")
russian_stopwords_new.extend(my_stopwords)
not_stopwords = {'не', 'ни'}
russian_stopwords = [word for word in russian_stopwords_new if word not in not_stopwords]

from sklearn.model_selection import  StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from matplotlib import pyplot as plt
from io import BytesIO, StringIO
from sklearn.model_selection import cross_validate
import pickle

import sys
print(sys.path)
import wandb

['/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/ML', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages', '/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification']


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [5]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [219]:
from data_preproc.src.utils import preprocess_text1, preprocess_text2, preprocess_text3, preprocess_text4, tokenizing

In [292]:
BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
# FILENAME = 'splitted_w_ds_df.csv'
seed = 42
model_name = 'tfidf_logreg_pipeline_'+'preproc4_'+str(seed)
test_size = 0.3

In [253]:
wandb.login()

True

In [254]:
wandb.init(project="authorship_identification", name=model_name, entity="mlds23_ai", tags=['baseline'])

In [255]:
wandb.config['random_state'] = seed

# Загружаем уже расспличенные тексты

In [293]:
session = boto3.session.Session()

In [294]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJErlaldUmioGbHQSqJ70MR',
    aws_secret_access_key='YCPSba_JgloNYSNWcnKO2CYCEB8PFR1Iwgr2jIUy',
    region_name='ru-cental1'
)

In [295]:
csv_obj = s3.get_object(Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)

In [296]:
df = pd.read_csv(StringIO(csv_obj['Body'].read().decode('utf-8')))

In [ ]:
#загружает в локальную директорию, потом отдельно надо считывать
# s3.download_file(Filename=FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)
# df = pd.read_csv('splitted_df.csv')

In [297]:
df.head()

,target,text,book
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt


In [301]:
df.text[0].split()

['Узкими',
 'горными',
 'тропинками',
 ',',
 'от',
 'одного',
 'дачного',
 'поселка',
 'до',
 'другого',
 ',',
 'пробиралась',
 'вдоль',
 'южного',
 'берега',
 'Крыма',
 'маленькая',
 'бродячая',
 'труппа',
 '.',
 'Впереди',
 'обыкновенно',
 'бежал',
 ',',
 'свесив',
 'набок',
 'длинный',
 'розовый',
 'язык',
 ',',
 'белый',
 'пудель',
 'Арто',
 ',',
 'остриженный',
 'наподобие',
 'льва',
 '.',
 'У',
 'перекрестков',
 'он',
 'останавливался',
 'и',
 ',',
 'махая',
 'хвостом',
 ',',
 'вопросительно',
 'оглядывался',
 'назад',
 '.',
 'По',
 'каким-то',
 'ему',
 'одному',
 'известным',
 'признакам',
 'он',
 'всегда',
 'безошибочно',
 'узнавал',
 'дорогу',
 'и',
 ',',
 'весело',
 'болтая',
 'мохнатыми',
 'ушами',
 ',',
 'кидался',
 'галопом',
 'вперед',
 '.',
 'За',
 'собакой',
 'шел',
 'двенадцатилетний',
 'мальчик',
 'Сергей',
 ',']

#  Выделяем классы метками

In [41]:
enc = LabelEncoder()
df['author_id'] = enc.fit_transform(df['target'])

In [42]:
#посмотрим на распределение классов
df.groupby('author_id').author_id.count().sort_values()

author_id
1     379
7     446
8     627
5     725
3     804
6    1047
4    1295
0    1299
9    2387
2    2706
Name: author_id, dtype: int64

In [43]:
labels = df['target']

# подготовка для TF-IDF

In [44]:
#удаляем цифры, http, приводим к нижнему регистру, убираем пробелы, короткие слова и пунктуацию+лемматизация
df['text_proc1'] = df['text'].apply(lambda x: preprocess_text1(x, tostr=True))
#удаляем цифры, http, приводим к нижнему регистру, убираем пробелы, короткие слова. ОСТАВЛЯЕМ пунктуацию+лемматизация
df['text_proc2'] = df['text'].apply(lambda x: preprocess_text2(x, tostr=True))
#удаляем цифры, http, приводим к нижнему регистру, убираем пробелы, пунктуацию ОСТАВЛЯЕМ короткие слова.+лемматизация
df['text_proc3'] = df['text'].apply(lambda x: preprocess_text3(x, tostr=True))
#удаляем цифры, http, приводим к нижнему регистру, убираем пробелы, пунктуацию ОСТАВЛЯЕМ короткие слова и стопслова +лемматизация
df['text_proc4'] = df['text'].apply(lambda x: preprocess_text4(x, tostr=True))

In [45]:
df.head()

,target,text,book,book_share,split,author_id,text_proc1,text_proc2,text_proc3,text_proc4
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,0.109838,test,6,узкий горный тропинка дачный поселок пробирать...,"узкий горный тропинка , дачный поселок , ...",узкий горный тропинка дачный поселок пробирать...,узкий горный тропинка от один дачный поселок д...
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,0.109838,test,6,свернутый ковер акробатический упражнение прав...,"свернутый ковер акробатический упражнение , ...",свернутый ковер акробатический упражнение прав...,свернутый ковер для акробатический упражнение ...
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,0.109838,test,6,позабывать кроме шарманка предательский труба ...,"позабывать кроме , шарманка предательский тр...",позабывать кроме шарманка предательский труба ...,позабывать кроме то быть в шарманка два предат...
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,0.109838,test,6,тайный грусть — поделать древний орган прос...,тайный грусть : — поделать древний орган ......,тайный грусть — поделать древний орган прос...,тайный грусть — что поделать древний орган ...
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,0.109838,test,6,ладно кормить сергей сей пора бог давать покор...,", ладно кормить , сергей , сей пора ,...",ладно кормить сергей сей пора бог давать покор...,да уж ладно кормить она мы с ты сергей до сей ...


# TF-IDF

## +wandb

In [256]:
def tfidf_model_with_cv(df):
    # russian_stopwords = russian_stopwords 
    max_features = 550
    min_df = 2
    # seed = seed 
    # test_size = test_size
    
    
    x_train, x_test, y_train, y_test = train_test_split(df['text_proc4'], df['author_id'], test_size = test_size, random_state=seed,\
                                                    stratify=df['author_id'])

    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, 
                             # max_features=max_features, min_df=min_df
                            )


    classifier = LogisticRegression(C=1e2, n_jobs=4, solver='sag',
                               random_state=seed, verbose=0,
                               multi_class='multinomial',
                               fit_intercept=True, max_iter=400)


    model = Pipeline([('tf_idf', tf_idf),
                    ('classifier', classifier)])


    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    

    wandb.config['dataset'] = FILENAME

    wandb.config['preprocessing'] = {
        'lowercase': True,
        'remove_punct': True,
        'remove_html_tags': True,
        'remove_numbers': True,
        'lemmatization': True,
        'remove_short_words': False,
        'remove_stop_words': False,
    }
    wandb.config['embedding'] = {
        'embedding_type': 'TF-IDF'    
    }

    wandb.config['classifier'] = {
        'classifier_name': model['classifier'].__class__.__name__,
        'solver': model['classifier'].solver,
        'max_iter': model['classifier'].max_iter,
        'C': model['classifier'].C,
        'n_jobs': model['classifier'].n_jobs,
        'multi_class': model['classifier'].multi_class,
        'fit_intercept': model['classifier'].fit_intercept,
        
        
    }
    

    wandb.config['evaluation'] = {
        'test_size': test_size,
        'train_cv': skf.n_splits
    }


    cv_results = cross_validate(model, x_train, y_train,
                                scoring=['f1_micro', 'f1_macro', 'f1_weighted'],
                                cv=skf)


    print("F1 Micro:", cv_results['test_f1_micro'].mean())
    print("F1 Macro:", cv_results['test_f1_macro'].mean())
    print("F1 Weighted:", cv_results['test_f1_weighted'].mean())
    

    metrics_train = dict()

    for metric in ['test_f1_micro', 'test_f1_macro', 'test_f1_weighted']:
        metrics_train[f'{metric[5:]}_mean'] = cv_results[metric].mean()
        metrics_train[f'{metric[5:]}_std'] = cv_results[metric].std()
        
    wandb.log({
    'train': metrics_train
    })
        
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    return model, metrics_train, y_pred 

In [257]:
model, metrics_train, y_pred   = tfidf_model_with_cv(df)

F1 Micro: 0.8934146341463416
F1 Macro: 0.8900527979995052
F1 Weighted: 0.8926013385041


In [258]:
model

Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(stop_words=['и', 'в', 'во', 'что', 'он', 'на',
                                             'я', 'с', 'со', 'как', 'а', 'то',
                                             'все', 'она', 'так', 'его', 'но',
                                             'да', 'ты', 'к', 'у', 'же', 'вы',
                                             'за', 'бы', 'по', 'только', 'ее',
                                             'мне', 'было', ...])),
                ('classifier',
                 LogisticRegression(C=100.0, max_iter=400,
                                    multi_class='multinomial', n_jobs=4,
                                    random_state=42, solver='sag'))])

In [259]:
metrics_train

{'f1_micro_mean': 0.8934146341463416,
 'f1_micro_std': 0.004838252298750583,
 'f1_macro_mean': 0.8900527979995052,
 'f1_macro_std': 0.006902788931788446,
 'f1_weighted_mean': 0.8926013385041,
 'f1_weighted_std': 0.005130526215383625}

In [19]:
def calc_metrics(y_test, y_pred, df):

    metrics_test = dict()

    for average in ['micro', 'weighted', 'macro']:
        metrics_test[f'f1_{average}'] = f1_score(y_test, y_pred, average=average)
        
    wandb.log({
    'test': metrics_test})

    precision_recall_f1_test_df = pd.DataFrame(
        precision_recall_fscore_support(y_test, y_pred),
        columns=df.author_id.unique().tolist(),
        index = ['precision_test', 'recall_test', 'f1_test', 'support']).T.drop(columns=['support']).reset_index()

    precision_recall_f1_test_df.rename(columns={'index': 'author'}, inplace=True)
    wandb.log({
    'precision_recall_f1_table_test': wandb.Table(dataframe=precision_recall_f1_test_df)})
    
    wandb.log({
    'conf_mat_test': wandb.plot.confusion_matrix(probs=None, y_true=y_test.tolist(), preds=y_pred, class_names=df.author_id.unique().tolist())   
})
    
    return metrics_test, precision_recall_f1_test_df


In [261]:
metrics_test, precision_recall_f1_test_df = calc_metrics(y_test, y_pred, df)

In [262]:
metrics_test

{'f1_micro': 0.9069701280227596,
 'f1_weighted': 0.9060874665840136,
 'f1_macro': 0.9032614957378018}

In [263]:
precision_recall_f1_test_df

,author,precision_test,recall_test,f1_test
0,6,0.968586,0.948718,0.958549
1,0,0.991071,0.973684,0.982301
2,7,0.886705,0.944581,0.914729
3,3,0.915094,0.804979,0.856512
4,1,0.909774,0.933162,0.921320
5,9,0.937853,0.764977,0.842640
6,5,0.888179,0.885350,0.886762
7,2,0.968254,0.910448,0.938462
8,4,0.899371,0.760638,0.824207
9,8,0.875325,0.941341,0.907133


In [264]:
wandb.finish()

## просто пайплайн без логирования

In [ ]:
# Попробуем добавить регуляризацию к логистической регрессии
# classifier = LogisticRegression(C=0.5, n_jobs=4, solver='sag', random_state=seed, 
verbose=0, multi_class='multinomial', fit_intercept=True, max_iter=400, penalty='l2')

# Попробуем уменьшить количество признаков для TF-IDF
# max_features = 300

# Попробуем уменьшить степень регуляризации
# C = 0.5

In [278]:
def tfidf_model_with_cv(df, split = True):
    # russian_stopwords = russian_stopwords 
    max_features = 500 
    min_df = 2
    # seed = seed 
    # test_size= test_size

    if not split: 
        x_train, x_test, y_train, y_test = train_test_split(df['text_proc4'], df['author_id'], test_size = test_size, random_state=seed,\
                                                    stratify=df['author_id'])

    else:
        x_test = df[df.split == 'test']['text_proc1']
        x_train = df[df.split == 'train']['text_proc1']
        y_test = df[df.split == 'test']['author_id']
        y_train = df[df.split == 'train']['author_id']

    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, 
                               max_features=max_features, 
                             min_df=min_df
                            )


    classifier = LogisticRegression(C=1e2, n_jobs=4, solver='sag',
                               random_state=seed, verbose=0,
                               multi_class='multinomial',
                               fit_intercept=True, max_iter=400,
                                   penalty='l2'
                                   )


    model = Pipeline([('tf_idf', tf_idf),
                    ('classifier', classifier)])


    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)


    cv_results = cross_validate(model, x_train, y_train,
                                scoring=['f1_micro', 'f1_macro', 'f1_weighted'],
                                cv=skf)


    print("F1 Micro:", cv_results['test_f1_micro'].mean())
    print("F1 Macro:", cv_results['test_f1_macro'].mean())
    print("F1 Weighted:", cv_results['test_f1_weighted'].mean())
    

    metrics_train = dict()

    for metric in ['test_f1_micro', 'test_f1_macro', 'test_f1_weighted']:
        metrics_train[f'{metric[5:]}_mean'] = cv_results[metric].mean()
        metrics_train[f'{metric[5:]}_std'] = cv_results[metric].std()
        
    # Обучение пайплайна на полных данных и прогноз
    model.fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    


    return model, metrics_train, y_pred_train, y_pred_test

In [279]:
model, metrics_train, y_pred_train, y_pred_test =  tfidf_model_with_cv(df)

F1 Micro: 0.6771458908913646
F1 Macro: 0.6305734687082104
F1 Weighted: 0.6769699378739163


In [280]:
metrics_train

{'f1_micro_mean': 0.6771458908913646,
 'f1_micro_std': 0.010364696149342937,
 'f1_macro_mean': 0.6305734687082104,
 'f1_macro_std': 0.010530161359530003,
 'f1_weighted_mean': 0.6769699378739163,
 'f1_weighted_std': 0.010592654773574963}

In [281]:
# model

In [282]:
# x_train, x_test, y_train, y_test = train_test_split(df['text_proc4'], df['author_id'], test_size = test_size, random_state=seed,\
#                                                     stratify=df['author_id'])

In [283]:
# preds_test = model.predict(x_test)

In [284]:
# if y_pred.all() == preds_test.all():
#     print("Массивы эквивалентны")

In [285]:
def calc_metrics(y_test, y_pred):

    metrics_test = dict()

    for average in ['micro', 'weighted', 'macro']:
        metrics_test[f'f1_{average}'] = f1_score(y_test, y_pred, average=average)

    precision_recall_f1_test_df = pd.DataFrame(
        precision_recall_fscore_support(y_test, y_pred),
        columns=df.author_id.unique().tolist(),
        index = ['precision_test', 'recall_test', 'f1_test', 'support']).T.drop(columns=['support']).reset_index()

    precision_recall_f1_test_df.rename(columns={'index': 'author'}, inplace=True)
    
    return metrics_test, precision_recall_f1_test_df

In [286]:
metrics_test, precision_recall_f1_test_df = calc_metrics(y_test, y_pred_test)

In [287]:
metrics_test

{'f1_micro': 0.3718473074301295,
 'f1_weighted': 0.375705659314937,
 'f1_macro': 0.3429279196304781}

In [288]:
precision_recall_f1_test_df

,author,precision_test,recall_test,f1_test
0,6,0.440994,0.445141,0.443058
1,0,0.343434,0.369565,0.356021
2,7,0.485294,0.424286,0.452744
3,3,0.253521,0.274112,0.263415
4,1,0.341246,0.347432,0.344311
5,9,0.226601,0.273810,0.247978
6,5,0.275974,0.332031,0.301418
7,2,0.376238,0.351852,0.363636
8,4,0.227027,0.270968,0.247059
9,8,0.429603,0.391447,0.409639


In [289]:
metrics_train, precision_recall_f1_train_df = calc_metrics(y_train, y_pred_train)

In [290]:
metrics_train

{'f1_micro': 0.8773488213187564,
 'f1_weighted': 0.8771457884961157,
 'f1_macro': 0.8816578109866402}

In [291]:
precision_recall_f1_train_df

,author,precision_test,recall_test,f1_test
0,6,0.866267,0.885714,0.875883
1,0,0.975265,0.961672,0.968421
2,7,0.877298,0.880359,0.878826
3,3,0.861301,0.828666,0.844668
4,1,0.857728,0.869295,0.863472
5,9,0.868030,0.838420,0.852968
6,5,0.832468,0.810367,0.821268
7,2,0.952663,0.952663,0.952663
8,4,0.875556,0.834746,0.854664
9,8,0.892114,0.915683,0.903745


## пиклим и загружаем в s3

In [ ]:
# вариант загрузки с локальным файлом
# with open('tfidf_logreg_pipeline.pkl', 'wb') as file:
#     pickle.dump((model), file)

In [ ]:
#вот так загружаем
# with open('tfidf_logreg_pipeline.pkl', 'rb') as file:
#     tfidf_logreg_pipeline = pickle.load(file)

In [ ]:
# DATA_PATH = '/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/ML/' #это локальный путь! Куда сохранился мой csv файл
# NEW_FILE_NAME = 'tfidf_logreg_pipeline.pkl'

In [ ]:
# s3.upload_file(f'{DATA_PATH}{NEW_FILE_NAME}', BUCKET_NAME, f'models/{NEW_FILE_NAME}')

In [75]:
pickle_buffer = BytesIO()
pickle_byte_obj = pickle.dump(model, pickle_buffer)

s3.put_object(Body=pickle_buffer.getvalue(), 
              Bucket=BUCKET_NAME, 
              Key=f'models/{model_name}.pkl')

{'ResponseMetadata': {'RequestId': 'dc5ccc9ad99dd59c',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Tue, 05 Dec 2023 18:05:50 GMT',
   'content-type': 'application/octet-stream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'etag': '"a53c37f532c5bfd235b15b1ede148c52"',
   'x-amz-request-id': 'dc5ccc9ad99dd59c'},
  'RetryAttempts': 0},
 'ETag': '"a53c37f532c5bfd235b15b1ede148c52"'}